# Project

## A Martial Arts Teacher wants to set up his office in Lucknow, Uttar Prradesh . He wants to set his office in the location where the crime rates are high(as people are more willing to learn martial arts) and also in the locality where people Visit more often like places near spas,gyms,resturants etc. 
## So In order to solve this problem we will be taking the following steps:
 ### 1.Taking the data base from police and checking which area has high crime Rates.
 ### 2.Then using our fourSquare API we will determine the best neighbor hood. 



In [1]:
#importing Libraries
import numpy as np   # library to handle data in a vectorized manner

import pandas as pd   # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print("Libraries Imported!")

     |████████████████████████████████| 112kB 1.1MB/s eta 0:00:01
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.1
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    attrs-21.2.0               |     pyhd8ed1ab_0          44 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    ca-certificates-2021.5.30  |       ha878542_0         136 KB  conda-forge
    certifi-2021.5.30          |   py36h5fab9bb_0         141 KB  con

In [2]:
df_area = pd.read_csv('sam_20.csv')

print('Dataset downloaded and read into a pandas dataframe!')

Dataset downloaded and read into a pandas dataframe!


In [3]:
df_area.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01-05-2016 00:00,23:50,Eastern,NONE,Lucknow Golf Course,80.952700,26.84150,"(37.7857883766888, -122.412970537591)",1.600000e+13
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,Eastern,"ARREST, BOOKED",Vaibhav Khand,80.948800,26.82130,"(26.8213,80.9488 )",1.200000e+13
2,141059263,WARRANTS,WARRANT ARREST,Monday,04/25/2016 12:00:00 AM,14:59,Northern,"ARREST, BOOKED",Lucknow Cantonment,80.948700,26.82140,"(37.7299809672996, -122.388856204292)",1.410000e+13
3,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01-05-2016 00:00,23:50,Eastern,NONE,Lucknow Golf Course,81.020311,26.86249,"(37.7857883766888, -122.412970537591)",1.600000e+13
4,160002740,NON-CRIMINAL,LOST PROPERTY,Friday,01-01-2016 00:00,00:30,Western,NONE,Barabanki,80.989996,26.85233,"(37.7650501214668, -122.419671780296)",1.600000e+13


In [4]:
df_area.rename(columns={'X':'Longitude'},inplace=True)
df_area.rename(columns={'Y':'Latitude'},inplace=True)

In [5]:
df_area.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,Longitude,Latitude,Location,PdId
0,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01-05-2016 00:00,23:50,Eastern,NONE,Lucknow Golf Course,80.952700,26.84150,"(37.7857883766888, -122.412970537591)",1.600000e+13
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,Eastern,"ARREST, BOOKED",Vaibhav Khand,80.948800,26.82130,"(26.8213,80.9488 )",1.200000e+13
2,141059263,WARRANTS,WARRANT ARREST,Monday,04/25/2016 12:00:00 AM,14:59,Northern,"ARREST, BOOKED",Lucknow Cantonment,80.948700,26.82140,"(37.7299809672996, -122.388856204292)",1.410000e+13
3,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01-05-2016 00:00,23:50,Eastern,NONE,Lucknow Golf Course,81.020311,26.86249,"(37.7857883766888, -122.412970537591)",1.600000e+13
4,160002740,NON-CRIMINAL,LOST PROPERTY,Friday,01-01-2016 00:00,00:30,Western,NONE,Barabanki,80.989996,26.85233,"(37.7650501214668, -122.419671780296)",1.600000e+13


In [6]:

limit=500
df_area=df_area.iloc[0:limit, :]

In [7]:
# finding the latitude and longitude of San Francisco
address= 'Uttar Pradesh , India'
geolocator=Nominatim(user_agent='area_explorer')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print(latitude,longitude)


27.1303344 80.859666


In [8]:
sanfran_map= folium.Map(location=[latitude,longitude],zoom_start=12)
sanfran_map

## Now we will place the data onto the map

In [9]:
incidents = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(df_area.Latitude, df_area.Longitude):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='green',
            fill=True,
            fill_color='red',
            fill_opacity=0.6
        )
    )

# add incidents to map
sanfran_map.add_child(incidents)

In [10]:
#lucknow_grouped_clustering = lucknow_grouped.drop('Neighborhood', 1)

# run k-means clustering
#kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lucknow_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10]

#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#lucknow_merged = lucknow_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
#lucknow_merged = lucknow_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#lucknow_merged.head()

# now we will display data in from of clusters so to get clear picture of the data 
from folium import plugins

# let's start again with a clean copy of the map of San Francisco
sanfran_map = folium.Map(location = [latitude, longitude], zoom_start = 12)

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(sanfran_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(df_area.Latitude, df_area.Longitude, df_area.Category):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)


sanfran_map

## Now we can clearly see the data on the map ,which area has high crime rate and which area has low crime rate 

In [11]:
df_area.groupby('Address')['PdDistrict'].count().sort_values(ascending=False)[:10]

Address
Lucknow Golf Course                                                     42
Mehboob Ganj, Lucknow, Uttar Pradesh 226003                             37
Ganga Darshan Colony                                                    33
Mozzam Nagar, Wazirbagh Mohallah, Lucknow, Uttar Pradesh 226003         33
Mohammadpur                                                             27
Ahiran Khera, Lucknow, Uttar Pradesh 226003                             26
Sector J, Sector-A, Sector L, Aliganj, Lucknow, Uttar Pradesh 226024    23
Preeti Nagar, Lucknow                                                   19
Sardarnagar, Lucknow, Uttar Pradesh 226003                              19
IIT Society Rd                                                          18
Name: PdDistrict, dtype: int64

#### Now this is a very important observation as now have the data from which we can determine crime areas

In [12]:
#we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
CLIENT_ID = 'SFKLP2QNSKQ4ZRTJN2TWN35H500O45ORSBVIZ1URMZLX4FBO' # your Foursquare ID
CLIENT_SECRET = 'Q2DEUR5VU50HB2BHGYMQDE5B0AQ2TAJCT0QJWD1D2IK4LVHL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SFKLP2QNSKQ4ZRTJN2TWN35H500O45ORSBVIZ1URMZLX4FBO
CLIENT_SECRET:Q2DEUR5VU50HB2BHGYMQDE5B0AQ2TAJCT0QJWD1D2IK4LVHL


In [13]:
# now we dont need that much data from the table
df_new=df_area[['PdDistrict','Address','Latitude','Longitude']]
df_new.groupby(['PdDistrict'])['Address','Latitude','Longitude'].apply(lambda x:', '.join(x)).sort_values(ascending=False).to_frame().reset_index()
df_new.head()


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,PdDistrict,Address,Latitude,Longitude
0,Eastern,Lucknow Golf Course,26.84150,80.952700
1,Eastern,Vaibhav Khand,26.82130,80.948800
2,Northern,Lucknow Cantonment,26.82140,80.948700
3,Eastern,Lucknow Golf Course,26.86249,81.020311
4,Western,Barabanki,26.85233,80.989996


## Now We have a table according to which we can find the neighborhood data

### Now using API to find near by venues

In [14]:
Latitude_1=df_new.loc[0,'Latitude']
Longitude_1=df_new.loc[0,'Longitude']
near_by_place=df_new.loc[0,'Address']
print('Latitude and longitude values of {} are {}, {}.'.format(near_by_place, 
                                                               Latitude_1, 
                                                               Longitude_1))

Latitude and longitude values of Lucknow Golf Course are 26.8415, 80.9527.


In [15]:
# lets check for venues which are under 500 meter radius
LIMIT=20
radius=800
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Latitude_1, 
    Longitude_1, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=SFKLP2QNSKQ4ZRTJN2TWN35H500O45ORSBVIZ1URMZLX4FBO&client_secret=Q2DEUR5VU50HB2BHGYMQDE5B0AQ2TAJCT0QJWD1D2IK4LVHL&v=20180605&ll=26.8415,80.9527&radius=800&limit=20'

In [16]:
results=requests.get(url).json()

In [17]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [18]:
#sorting data
venues=results['response']['groups'][0]['items']
nearby_venues=json_normalize(venues)

filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories']=nearby_venues.apply(get_category_type , axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Pizza Hut,Pizza Place,26.843773,80.947115
1,Rovers,Café,26.845948,80.946365
2,Lucknow Zoo,Zoo,26.845506,80.952115
3,Walnut World Bakery,Bakery,26.845635,80.947731


In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [20]:
nearby_venues

,name,categories,lat,lng
0,Pizza Hut,Pizza Place,26.843773,80.947115
1,Rovers,Café,26.845948,80.946365
2,Lucknow Zoo,Zoo,26.845506,80.952115
3,Walnut World Bakery,Bakery,26.845635,80.947731


## Now super imposing the venues in the map to get clear picture and our client can clearly visualize the data.  

In [21]:
sanfran_new_map=folium.Map(location=[26.862345,80.85966],zoom_start=15)    # 37.7790262 -122.4199061
sanfran_new_map

In [22]:
incidents_1 = folium.map.FeatureGroup()


for lat_1, lng_1, in zip(nearby_venues.lat, nearby_venues.lng):
    incidents_1.add_child(
        folium.features.CircleMarker(
            [lat_1, lng_1],
            radius=8, # define how big you want the circle markers to be
            color='red',
            fill=True,
            fill_color='yellow',
            fill_opacity=0.6
        )
    )

# add incidents to map
sanfran_new_map.add_child(incidents_1)

# We have shown  our customer the most appropriate place with near by venues
